In [12]:
import numpy as np

# Semana 2 PIBIC

## Gerando Tabelas Verdade (Truth Tables)

Para a geração de tabelas verdade com n inputs no circuito, com $n \ge 1$. Esta é uma função recursiva que gera uma tabela verdade em forma de matriz para n inputs arbitrários.

Talvez existam formas mais rápidas e menores de fazê-las, mas esta função é mais legível para os usuários e para os parceiros da pesquisa. Modifiquei o nome das variáveis para auxiliar no processo.

Além disso, modifiquei-a para que retornasse um numpy array ao invés de um array convencional do python, aumentando a eficiência da função visto que numpy arrays são mais rápidos de se fazer modificações na função e até mesmo posteriores.

In [49]:
"""
DOCUMENTAÇÃO DA FUNÇÃO:

Retorna uma tabela verdade (truth_table) dado um circuito com n inputs maiores ou iguais a 1
Checa se a variável number_of_inputs é inteira. Se sim, continua normalmente, se não lança uma exceção com a mensagem
'Parameter \'number_of_inputs\' must be a integer'

:number_of_inputs: número de entradas do circuito. Ditarão a quantidade de colunas da tabela verdade
"""

def TruthTable(NumberOfInputs):
    if not isinstance(NumberOfInputs, int):
        raise ValueError('Parameter \'number_of_inputs\' must be a integer') 
    if NumberOfInputs < 1:
        return [[]]
    
    SubTable = TruthTable(NumberOfInputs - 1)
    return [row + [v] for row in SubTable for v in [0,1]]

In [50]:
table = np.array(TruthTable(3))
table

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]])

## Modulação PAM

Quando tratamos de tabelas verdade de circuitos eletrônicos, muitas vezes precisamos converter esses sinais em Modulação por Amplitude do Pulso (PAM: Pulse Amplitude Moduling) para realizarmos cálculos ou verificações e depois retornamos a representação do circuito a meneira original, com 0's e 1's.

Para isso, convertemos os campos (labels) das tabelas verdades por $1+\epsilon$ caso o valor no campo seja 1 ou $1-\epsilon$ caso seja 0. A matriz resultante é chamada de $\textbf{Imp}$.

No exemplo a seguir, utilizaremos a variável table definida no código acima para os testes e o valor $\epsilon = 5$:

In [51]:
"""
DOCUMENTAÇÃO DA FUNÇÃO:

Função que retorna uma modulação PAM para uma dada tabela verdade e certo epsilon real arbitrário
Não modifica a função original, realiza alterações em uma cópia e a retorna no final
Functiona para qualquer tabela verdade (truth_table) com n inputs

:truth_table: tabela verdade a ser analisada
:epsilon: epsilon no qual os valore serão calculados
"""

def PAM(Truthtable, epsilon):
    PamTable = Truthtable.copy()
    rows, columns = PamTable.shape
    for row in range(rows):
        for column in range(columns):
            if PamTable[row][column] == 1:
                PamTable[row][column] = 1 + epsilon
            else:
                PamTable[row][column] = 1 - epsilon
    return PamTable

In [52]:
Imp = PAM(table, 5)
print(f'IMP: \n{Imp}\n\nTABELA VERDADE ORIGINAL: \n{table}')

IMP: 
[[-4 -4 -4]
 [-4 -4  6]
 [-4  6 -4]
 [-4  6  6]
 [ 6 -4 -4]
 [ 6 -4  6]
 [ 6  6 -4]
 [ 6  6  6]]

TABELA VERDADE ORIGINAL: 
[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]


Note que a linha de código dada por: 

``PAM_table = truth_table.copy()``

faz com que a variável PAM_table seja uma cópia da truth_table, não alterando assim a tabela verdade inicial recebdia como parãmetro. Isso é importante pois podemos realizar os cálculos necessários com a variável Imp que guarda o valor de retorno da função sem perder a tabela verdade original (Desfaz a necessidade da função $\textbf{InvPAM}$).

## Geração de Tabelas Verdade Padrões (OR, AND)

Para dar continuidade à pesquisa, precisamos de uma função que nos retorne tabelas verdade de operadores lógicos padrões (OR, AND etc...).

In [54]:
"""
DOCUMENTAÇÃO DA FUNÇÃO:

Retorna uma tabela verdade do circuito lógico OR ou AND de acordo com uma tabela verdade (truth_table) e quantidade
de inputs (number_of_inputs). Checa se a tabela está vazia, se sim a retorna (vazia), se não realiza os cálculos.

params: truth_table, number_of_inputs, boolean_expression (OR, AND)
"""

def OrAndTruthTableGenerator(TruthTable, BooleanOperator):
    rows, columns = TruthTable.shape
    BooleanOperator = BooleanOperator.lower()
    ResultTable = np.zeros(shape=(rows, 1), dtype=int)
    
    if TruthTable.size == 0:
        return TruthTable
    
    if BooleanOperator == 'or':
        for row in range(rows):
            BooleanRowArray = TruthTable[row][:]
            if np.any(BooleanRowArray):
                ResultTable[row][0] = 1
    elif BooleanOperator == 'and':
        for row in range(rows):
            BooleanRowArray = TruthTable[row][:]
            if np.all(BooleanRowArray):
                ResultTable[row][0] = 1;
    else:
        raise ValueError('Parameter boolean_expression must be \'or\' or \'and\'')
        
    return ResultTable

In [56]:
OR = OrAndTruthTableGenerator(table, 'or')
AND = OrAndTruthTableGenerator(table, 'and')
print(f'OR TRUTH TABLE: \n{OR} \n\nAND TRUTH TABLE: \n{AND}')

OR TRUTH TABLE: 
[[0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]] 

AND TRUTH TABLE: 
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]]


Perceba que os exemplos utilizados para os cálculos das funções são feitos utilizando tabelas verdade de 3 inputs. No entanto, como já documentado, essas funções aceitam tabelas verdade de n inputs, sendo n um número natural.